In [11]:
import pandas as pd
import numpy as np
import os
#!pip install mlxtend
from mlxtend.frequent_patterns import apriori, association_rules
#!pip install streamlit
import streamlit as st

In [12]:
data = pd.read_csv("risk_factors_cervical_cancer.csv")
data

,Age,Number of sexual partners,First sexual intercourse,Num of pregnancies,Smokes,Smokes (years),Smokes (packs/year),Hormonal Contraceptives,Hormonal Contraceptives (years),IUD,...,STDs: Time since first diagnosis,STDs: Time since last diagnosis,Dx:Cancer,Dx:CIN,Dx:HPV,Dx,Hinselmann,Schiller,Citology,Biopsy
0,18,4.0,15.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,?,?,0,0,0,0,0,0,0,0
1,15,1.0,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,?,?,0,0,0,0,0,0,0,0
2,34,1.0,?,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,?,?,0,0,0,0,0,0,0,0
3,52,5.0,16.0,4.0,1.0,37.0,37.0,1.0,3.0,0.0,...,?,?,1,0,1,0,0,0,0,0
4,46,3.0,21.0,4.0,0.0,0.0,0.0,1.0,15.0,0.0,...,?,?,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
853,34,3.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,?,?,0,0,0,0,0,0,0,0
854,32,2.0,19.0,1.0,0.0,0.0,0.0,1.0,8.0,0.0,...,?,?,0,0,0,0,0,0,0,0
855,25,2.0,17.0,0.0,0.0,0.0,0.0,1.0,0.08,0.0,...,?,?,0,0,0,0,0,0,1,0
856,33,2.0,24.0,2.0,0.0,0.0,0.0,1.0,0.08,0.0,...,?,?,0,0,0,0,0,0,0,0


In [13]:
def count_missing_values(datacol):
    return datacol.value_counts()["?"]

In [14]:
def str_to_numeric(data): # data should be with type objekt! A Dataframe column. 
    i_elements = []
    for i in data.to_list():
        if i == "?":
            i = 0
        i_elements.append(i)
    return pd.to_numeric(i_elements)

In [15]:
def num_partner_to_binary(datacol):
    j_elements = []
    datacol = str_to_numeric(datacol)
    #type(data["Number of sexual partners"])

    #data_list = data.to_list()
    for i in range(datacol.size):
        if datacol[i]/(data.at[i, "Age"]-10) >= 0.2:  # In average every five year a new partner from 10 years old.
            i = 1
        else:
            i = 0
        j_elements.append(i)
    datacol = j_elements
    return datacol


In [16]:
data["Number of sexual partners"] = num_partner_to_binary(data["Number of sexual partners"])
data["Number of sexual partners"]
print(data["Number of sexual partners"].sum())

277


In [17]:
def age_to_binary(data):
    i_elements = []
    for i in data.to_list():
        if i >= 35:  # Half of the average female age in Venezuela in 2017 is 35
            i = 1
        else:
            i = 0
        i_elements.append(i)
    return pd.to_numeric(i_elements)

In [18]:
data["Age"] = age_to_binary(data["Age"])
print(data["Age"].sum())

153


In [37]:
subdata_2 = data[["Age", "Number of sexual partners", "IUD", "STDs", 
               "Smokes", "STDs:HPV", "Hormonal Contraceptives"]]
subdata_2 = pd.DataFrame(subdata_2)
subdata_2.insert(0, "Cervical Positiv", [1 for i in range(858)])
#subdata_2.astype(bool)
for col in subdata_2.columns:
    subdata_2[col] = str_to_numeric(subdata_2[col])
    subdata_2[col] = subdata_2[col].astype(int)
#subdata_2 = subdata_2.astype(int)
subdata_2

,Cervical Positiv,Age,Number of sexual partners,IUD,STDs,Smokes,STDs:HPV,Hormonal Contraceptives
0,1,0,1,0,0,0,0,0
1,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0
3,1,1,0,0,0,1,0,1
4,1,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...
853,1,0,0,0,0,0,0,0
854,1,0,0,0,0,0,0,1
855,1,0,0,0,0,0,0,1
856,1,0,0,0,0,0,0,1


In [39]:
cwd = os.getcwd()
subdata_2.to_csv(cwd+"/subdata_2.csv")
frequent_itemsets_2 = apriori(subdata_2, min_support = 0.1, use_colnames = True)
frequent_itemsets_2["length"] = frequent_itemsets_2["itemsets"].apply(lambda x:len(x))
frequent_itemsets_2

F:\Anaconda\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets,length
0,1.000000,(Cervical Positiv),1
1,0.178322,(Age),1
2,0.322844,(Number of sexual partners),1
3,0.143357,(Smokes),1
4,0.560606,(Hormonal Contraceptives),1
5,0.178322,"(Age, Cervical Positiv)",2
6,0.322844,"(Cervical Positiv, Number of sexual partners)",2
7,0.143357,"(Cervical Positiv, Smokes)",2
8,0.560606,"(Hormonal Contraceptives, Cervical Positiv)",2
9,0.115385,"(Age, Hormonal Contraceptives)",2


In [49]:
rules = association_rules(frequent_itemsets_2, metric="lift", min_threshold=1.1)
rules["antecedents_length"] = rules["antecedents"].apply(lambda x: len(x))
rules["consequents_length"] = rules["consequents"].apply(lambda x: len(x))
rules.sort_values("lift",ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedents_length,consequents_length
0,(Age),(Hormonal Contraceptives),0.178322,0.560606,0.115385,0.647059,1.154213,0.015416,1.244949,1,1
1,(Hormonal Contraceptives),(Age),0.560606,0.178322,0.115385,0.205821,1.154213,0.015416,1.034626,1,1
2,"(Age, Cervical Positiv)",(Hormonal Contraceptives),0.178322,0.560606,0.115385,0.647059,1.154213,0.015416,1.244949,2,1
3,"(Hormonal Contraceptives, Cervical Positiv)",(Age),0.560606,0.178322,0.115385,0.205821,1.154213,0.015416,1.034626,2,1
4,(Age),"(Hormonal Contraceptives, Cervical Positiv)",0.178322,0.560606,0.115385,0.647059,1.154213,0.015416,1.244949,1,2
5,(Hormonal Contraceptives),"(Age, Cervical Positiv)",0.560606,0.178322,0.115385,0.205821,1.154213,0.015416,1.034626,1,2


Here we can see all rules have the same lift and komplementar confidence. 

In [50]:
rules.sort_values("confidence",ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedents_length,consequents_length
0,(Age),(Hormonal Contraceptives),0.178322,0.560606,0.115385,0.647059,1.154213,0.015416,1.244949,1,1
2,"(Age, Cervical Positiv)",(Hormonal Contraceptives),0.178322,0.560606,0.115385,0.647059,1.154213,0.015416,1.244949,2,1
4,(Age),"(Hormonal Contraceptives, Cervical Positiv)",0.178322,0.560606,0.115385,0.647059,1.154213,0.015416,1.244949,1,2
1,(Hormonal Contraceptives),(Age),0.560606,0.178322,0.115385,0.205821,1.154213,0.015416,1.034626,1,1
3,"(Hormonal Contraceptives, Cervical Positiv)",(Age),0.560606,0.178322,0.115385,0.205821,1.154213,0.015416,1.034626,2,1
5,(Hormonal Contraceptives),"(Age, Cervical Positiv)",0.560606,0.178322,0.115385,0.205821,1.154213,0.015416,1.034626,1,2


Because it makes no sense that the age to be consequents, so we remove the rules where age is in the consequents. 

In [57]:
rules[~rules["consequents"].str.contains("Age", regex=False)].sort_values("confidence", ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedents_length,consequents_length
0,(Age),(Hormonal Contraceptives),0.178322,0.560606,0.115385,0.647059,1.154213,0.015416,1.244949,1,1
2,"(Age, Cervical Positiv)",(Hormonal Contraceptives),0.178322,0.560606,0.115385,0.647059,1.154213,0.015416,1.244949,2,1
4,(Age),"(Hormonal Contraceptives, Cervical Positiv)",0.178322,0.560606,0.115385,0.647059,1.154213,0.015416,1.244949,1,2


Sort with lift:

In [59]:
rules[~rules["consequents"].str.contains("Age", regex=False)].sort_values("lift",ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedents_length,consequents_length
0,(Age),(Hormonal Contraceptives),0.178322,0.560606,0.115385,0.647059,1.154213,0.015416,1.244949,1,1
2,"(Age, Cervical Positiv)",(Hormonal Contraceptives),0.178322,0.560606,0.115385,0.647059,1.154213,0.015416,1.244949,2,1
4,(Age),"(Hormonal Contraceptives, Cervical Positiv)",0.178322,0.560606,0.115385,0.647059,1.154213,0.015416,1.244949,1,2


The frequency of Hepatitis B is 1 case per 100,000 (data from https://www.cdc.gov/hepatitis/hbv/hbvfaq.htmin year 2018)

The frequency of AIDs in Venezuela estimation from 2016: 0.6%